# Composition of agents as tool

Agents and hierarchy of agents, easily implemented with AutoGen, can create powerful abstraction and enable high degree of resuability.

In this example, we will create a multi-agent group chat to search for ETFs that satisfy a given investment direction and risk profile. There will be three agents:
- `user_proxy` will ask for relevant ETFs that satisfy an investment direction and risk profile
- `etf_retriever` will be equipped with the tool we created above.
- `risk_evaluator` will evaluate risk level of each of the proposed ETFs

In [ ]:
from typing import List, Dict
from xentropy.tool import Tool
from dotenv import load_dotenv
import os
import autogen

load_dotenv()

def termination(x):
    return x.get('name') == 'risk_evaluator'

# Load the etf search tool
# Check out how this tool is created in docs/rag-tool
etf_search = Tool.load('xentropy--etf_search', api_key=os.environ.get('XENTROPY_API_KEY'))

# we need more context length to complete this example
config_list = autogen.config_list_from_models(
    model_list=["gpt-35-16"],
)

llm_config = {
    "functions": [
        # added the new tool here
        {
            "name": etf_search.name,
            "description": etf_search.description,
            "parameters": etf_search.input_model_schema(),
        },
    ],
    "config_list": config_list,
    "request_timeout": 120,
}

# Construct the agent with the new config
etf_researcher = autogen.AssistantAgent(
    name="etf_researcher",
    system_message="You are an export in ETF research. You can use the functions you have been provided with.",
    llm_config=llm_config,
    max_consecutive_auto_reply=3,
    function_map={
        etf_search.name: etf_search.run,
    },
    is_termination_msg=termination
)


user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    system_message="Reply TERMINATE when the task is done.",
    max_consecutive_auto_reply=3,
    llm_config={
        "config_list": config_list,
    },
    is_termination_msg=termination
)

risk_evaluator = autogen.AssistantAgent(
    name="risk_evaluator",
    system_message="You are a risk management expert and specialise in evaluating risk associated with ETFs.",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=3,
    llm_config={
        "config_list": config_list,
    },
    is_termination_msg=termination
)

groupchat = autogen.GroupChat(
    func_call_filter=True,
    agents=[user_proxy, etf_researcher, risk_evaluator],
    messages=[],
    max_round=12
)
manager = autogen.GroupChatManager(
    groupchat=groupchat,
    llm_config=llm_config,
    is_termination_msg=termination
)

In [ ]:
user_proxy.initiate_chat(
    manager,
    message=f"""Search ETFs that invest in semiconductor. 
Evaluate the result by its risk level. 
Then, create a report to compare their historical performance and analysis their risk level."""
)

In [ ]:
from pydantic import BaseModel

class ETFReport(BaseModel):
    name:str
    industry:str
    symbol:str
    expense_ratio:float
    volitility:str
    risk_exposure:str
    top_holdings:List[str]
    performance:Dict

class ETFReportList(BaseModel):
    result:List[ETFReport]

In [ ]:
# Creating a structured output will ease integration with other systems

import json
from autogen import OpenAIWrapper

client = OpenAIWrapper(config_list=config_list)

conversation = json.dumps(list(manager.chat_messages.items())[0][1])

schema = ETFReportList.model_json_schema()

def extract_json_body(client, model, conversation, num_try):
    iteration = 0

    while iteration < num_try:

        raw_text = client.create(
            messages=[
                {
                    'role':'user',
                    'content':f"You will return a JSON object according this json schema: {schema} \n\n Based on the following conversation:{conversation} \n\nReturn the JSON object only.",
                }
            ],
        ).choices[0].message.content

        try:
            return model.model_validate_json(raw_text)
        except:
            pass

    raise Exception()


result = extract_json_body(client, ETFReportList, conversation, 3)

In [ ]:
from rich import print as rich_print

rich_print(result)